In [1]:
import pandas as pd
import os
import glob
import numpy as np
import time
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
from langdetect import DetectorFactory, detect
DetectorFactory.seed = 0
import re

def get_path(country, week):
    base = f'../../data/03-experiment/{country}/'
    path_tw = base + f'treatment/followers/00-raw/tweets/{week}/'
    rand = f'../../data/02-randomize/{country}/04-stratification/integrate/followers_randomized.parquet'
    baseline = base + 'baseline/00-raw/followers/tweets/'
    agg = base + f'treatment/followers/01-preprocess/'
    agg_base = base + 'baseline/01-preprocess/followers/'
    return path_tw, base, rand, baseline, agg, agg_base

df_vars = ['id', 'handle', 'author_id', 'created_at', 'text', 'lang', 'referenced_tweets',
           'entities.urls','public_metrics.like_count', 'public_metrics.quote_count', 
           'public_metrics.reply_count', 'public_metrics.retweet_count']

import sys
sys.path.insert(0, '../../src/utils')
from funcs import PreprocessTweets
def compute_engagement_tw(engage):
    engage["total_reactions"] = engage[
        [x for x in engage.columns if "public_metrics" in x]
    ].sum(axis="columns")
    engage["total_comments"] = engage[
        ["public_metrics.reply_count", "public_metrics.quote_count"]
    ].sum(axis="columns")
    engage.rename(
        {"public_metrics.retweet_count": "total_shares"}, axis=1, inplace=True
    )
    return(engage)

def lang_detect_na(tweet):
    try:
        lang = detect(tweet)
    except:
        lang = 'NA'
    return lang

def compute_engagement_tw2(engage):
    engage["total_reactions"] = engage[
        ["reply_count", "quote_count", "like_count", 
         'retweet_count']].sum(axis="columns")
    engage["total_comments"] = engage[
        ["reply_count", 
         "quote_count"]].sum(axis="columns")
    engage.rename(
        {"retweet_count": "total_shares"}, axis=1, inplace=True)
    return(engage)

def has_items_tw(df):

    has_items = PreprocessTweets(df).preprocess()
    has_items.drop(['entities.urls', 'description', 'display_url',
    'end', 'expanded_url', 'images', 'media_key', 'start', 
    'status', 'title', 'unwound_url', 'url'], axis=1, inplace=True)
    
    return(has_items)

### Aggregating baseline posts

In [5]:
### 
path_tw, base, rand, baseline, agg, agg_base = get_path('KE', 'december1')
onlyfiles = [f for f in listdir(baseline) if isfile(join(baseline, f))]
onlyfiles = [f.replace('.parquet', '') for f in onlyfiles]
onlyfiles.sort()
len(onlyfiles)

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: '../../data/03-experiment/KE/baseline/00-raw/followers/tweets/'

In [ ]:
for i in range(0, 83):
    df_final = pd.DataFrame()
    time.sleep(5)
    a = 1000*i 
    b = 1000*(i+1)
    for e in tqdm(onlyfiles[a:b]):
        df = pd.read_parquet(f'{baseline}{e}.parquet')
        df_final = pd.concat([df_final, df])
    df_final = df_final[df_vars]
    df_final = df_final.reset_index(drop=True)
    print(i, a, b)
    df_final.to_parquet(f'{agg_base}intermediate/baseline_{i}.parquet.gzip', compression = 'gzip')
    
df_final = pd.DataFrame()
for i in tqdm(onlyfiles[82000:]):
    df = pd.read_parquet(f'{baseline}{i}.parquet')
    df_final = pd.concat([df_final, df])

df_final = df_final[df_vars]
df_final = df_final.reset_index(drop=True)
df_final.to_parquet(f'{agg_base}intermediate/baseline_83.parquet.gzip', compression = 'gzip')

 32%|█████████████████████████▏                                                     | 319/1000 [00:12<00:38, 17.83it/s]

### SA

In [2]:
path_tw, base, rand, baseline, agg, agg_base = get_path('SA', 'december1')
onlyfiles = [f for f in listdir(baseline) if isfile(join(baseline, f))]
onlyfiles = [f.replace('.parquet', '') for f in onlyfiles]
onlyfiles.sort()
len(onlyfiles)

36803

In [3]:
for i in range(0, 74):
    df_final = pd.DataFrame()
    a = 500*i 
    b = 500*(i+1)
    time.sleep(5)
    for e in tqdm(onlyfiles[a:b]):
        time.sleep(0.1)
        df = pd.read_parquet(f'{baseline}{e}.parquet')
        df_final = pd.concat([df_final, df])
    df_final = df_final[df_vars]
    df_final = df_final.reset_index(drop=True)
    df_final = compute_engagement_tw(df_final)
    df_final = has_items_tw(df_final)
    print(i, a, b)
    df_final.to_parquet(f'{agg_base}intermediate/baseline_{i}.parquet.gzip', 
                        compression = 'gzip')

 36%|███▌      | 179/500 [00:26<00:50,  6.35it/s]

: 

: 

In [8]:
df_final = pd.DataFrame()
for i in tqdm(onlyfiles[10000:]):
    df = pd.read_parquet(f'{baseline2}{i}.parquet')
    df = df[df['created_at'] > '2022-08-03T00:00:00Z'].reset_index(drop=True)
    df_final = pd.concat([df_final, df])

df_final = df_final[df_vars]
df_final = compute_engagement_tw(df_final)
df_final = has_items_tw(df_final)
df_final = df_final.reset_index(drop=True)
df_final.to_parquet(f'{agg_base}intermediate_abs/baseline_25.parquet.gzip', compression = 'gzip')

100%|██████████| 87/87 [00:00<00:00, 96.83it/s] 


### Aggregate Treatment Posts (TweetPle Posts)

In [3]:
path_tw, base, rand, baseline, agg, agg_base = get_path('KE', 'april1')
onlyfiles = [f for f in listdir(path_tw) if isfile(join(path_tw, f))]
onlyfiles = [f.replace('.parquet', '') for f in onlyfiles]
onlyfiles.sort()
len(onlyfiles)

38632

In [ ]:
for i in range(51, 59):
    df_final = pd.DataFrame()
    a = 1000*i 
    b = 1000*(i+1)
    time.sleep(5)
    for e in tqdm(onlyfiles[a:b]):
        df = pd.read_parquet(f'{path_tw}{e}.parquet')
        df_final = pd.concat([df_final, df])
    df_final = df_final[df_vars]
    df_final = df_final.reset_index(drop=True)
    df_final = compute_engagement_tw(df_final)
    df_final = has_items_tw(df_final)
    print(i, a, b)
    df_final.to_parquet(f'{agg}intermediate/march_{i}.parquet.gzip', 
                        compression = 'gzip')

In [4]:
for i in range(31, 39):
    df_final = pd.DataFrame()
    a = 1000*i 
    b = 1000*(i+1)
    time.sleep(5)
    for e in tqdm(onlyfiles[a:b]):
        df = pd.read_parquet(f'{path_tw}{e}.parquet')
        df_final = pd.concat([df_final, df])
    df_final = df_final[df_vars]
    df_final = df_final.reset_index(drop=True)
    df_final = compute_engagement_tw(df_final)
    df_final = has_items_tw(df_final)
    print(i, a, b)
    df_final.to_parquet(f'{agg}intermediate/april_{i}.parquet.gzip', 
                        compression = 'gzip')

100%|██████████| 1000/1000 [00:24<00:00, 41.06it/s]


31 31000 32000


100%|██████████| 1000/1000 [00:22<00:00, 43.90it/s]


32 32000 33000


100%|██████████| 1000/1000 [00:18<00:00, 53.60it/s]


...

36 36000 37000


100%|██████████| 1000/1000 [00:15<00:00, 65.33it/s]


37 37000 38000


100%|██████████| 632/632 [00:09<00:00, 64.62it/s] 


38 38000 39000


In [5]:
path_tw, base, rand, baseline, agg, agg_base = get_path('SA', 'april2')
onlyfiles = [f for f in listdir(path_tw) if isfile(join(path_tw, f))]
onlyfiles = [f.replace('.parquet', '') for f in onlyfiles]
onlyfiles.sort()
len(onlyfiles)

29688

In [6]:
for i in range(0, 10):
    df_final = pd.DataFrame()
    a = 3000*i 
    b = 3000*(i+1)
    #time.sleep(5)
    for e in tqdm(onlyfiles[a:b]):
        df = pd.read_parquet(f'{path_tw}{e}.parquet')
        df_final = pd.concat([df_final, df])
    df_final = df_final.reset_index(drop=True)
    df_final['text'] =df_final['text'].astype(str)
    df_final['lang'] = df_final['text'].apply(lang_detect_na)
    df_final = compute_engagement_tw2(df_final)
    print(i, a, b)
    df_final.to_parquet(f'{agg}intermediate/april2_{i}.parquet.gzip', 
                        compression = 'gzip')

100%|██████████| 3000/3000 [00:47<00:00, 62.68it/s]


0 0 3000


100%|██████████| 3000/3000 [00:59<00:00, 50.76it/s]


1 3000 6000


100%|██████████| 3000/3000 [01:04<00:00, 46.23it/s]


...

7 21000 24000


100%|██████████| 3000/3000 [00:57<00:00, 51.75it/s]


8 24000 27000


100%|██████████| 2688/2688 [00:43<00:00, 62.45it/s]


9 27000 30000


In [2]:
path_tw, base, rand, baseline, agg, agg_base = get_path('SA', 'post_treat2')
onlyfiles = [f for f in listdir(path_tw) if isfile(join(path_tw, f))]
onlyfiles = [f.replace('.parquet', '') for f in onlyfiles]
onlyfiles.sort()
len(onlyfiles)

29010

In [3]:
for i in tqdm(range(7, 10)):
    df_final = pd.DataFrame()
    a = 3000*i 
    b = 3000*(i+1)
    #time.sleep(5)
    for e in onlyfiles[a:b]:
        df = pd.read_parquet(f'{path_tw}{e}.parquet')
        df_final = pd.concat([df_final, df])
    df_final = df_final.reset_index(drop=True)
    df_final['text'] =df_final['text'].astype(str)
    df_final['lang2'] = df_final['text'].apply(lang_detect_na)
    df_final = compute_engagement_tw2(df_final)
    print(i, a, b)
    df_final.to_parquet(f'{agg}intermediate/posttreat2_{i}.parquet.gzip', 
                        compression = 'gzip')

  0%|          | 0/3 [00:00<?, ?it/s]

7 21000 24000


 33%|███▎      | 1/3 [53:08<1:46:16, 3188.21s/it]

8 24000 27000


 67%|██████▋   | 2/3 [1:22:47<39:19, 2359.15s/it]

9 27000 30000


100%|██████████| 3/3 [1:41:25<00:00, 2028.51s/it]


In [9]:
path_tw, base, rand, baseline, agg, agg_base = get_path('SA', 'april')
onlyfiles = [f for f in listdir(path_tw) if isfile(join(path_tw, f))]
onlyfiles = [f.replace('.parquet', '') for f in onlyfiles]
onlyfiles.sort()
len(onlyfiles)

29560

In [10]:
for i in tqdm(range(0, 10)):
    df_final = pd.DataFrame()
    a = 3000*i 
    b = 3000*(i+1)
    #time.sleep(5)
    for e in onlyfiles[a:b]:
        df = pd.read_parquet(f'{path_tw}{e}.parquet')
        df_final = pd.concat([df_final, df])
    df_final = df_final.reset_index(drop=True)
    df_final['text'] =df_final['text'].astype(str)
    df_final['lang2'] = df_final['text'].apply(lang_detect_na)
    df_final = compute_engagement_tw2(df_final)
    print(i, a, b)
    df_final.to_parquet(f'{agg}intermediate/april1_good{i}.parquet.gzip', 
                        compression = 'gzip')

  0%|          | 0/10 [00:00<?, ?it/s]

0 0 3000


 10%|█         | 1/10 [42:25<6:21:46, 2545.15s/it]

1 3000 6000


 20%|██        | 2/10 [1:13:18<4:45:05, 2138.18s/it]

...

 80%|████████  | 8/10 [3:38:48<47:34, 1427.24s/it]  

8 24000 27000


 90%|█████████ | 9/10 [4:02:21<23:42, 1422.96s/it]

9 27000 30000


100%|██████████| 10/10 [4:21:22<00:00, 1568.28s/it]


In [6]:
country = 'SA'
path_tw, base, rand, baseline, agg, agg_base = get_path('KE', 'april')
path_tw1 = f'../../data/03-experiment/{country}/treatment/followers/00-raw/tweets_batch2/may/'
onlyfiles = [f for f in listdir(path_tw1) if isfile(join(path_tw1, f))]
onlyfiles = [f.replace('.parquet', '') for f in onlyfiles]
onlyfiles.sort()
len(onlyfiles)

19178

In [2]:
country = 'SA'
path_tw, base, rand, baseline, agg, agg_base = get_path(country, 'april')
path_tw1 = f'../../data/03-experiment/{country}/treatment/followers/00-raw/tweets_batch2/may/'
onlyfiles = [f for f in listdir(path_tw1) if isfile(join(path_tw1, f))]
onlyfiles = [f.replace('.parquet', '') for f in onlyfiles]
onlyfiles.sort()

for i in tqdm(range(5, 7)):
    df_final = pd.DataFrame()
    a = 3000*i 
    b = 3000*(i+1)
    #time.sleep(5)
    for e in onlyfiles[a:b]:
        df = pd.read_parquet(f'{path_tw1}{e}.parquet')
        df_final = pd.concat([df_final, df])
    df_final = df_final.reset_index(drop=True)
    df_final['text'] =df_final['text'].astype(str)
    df_final['lang2'] = df_final['text'].apply(lang_detect_na)
    df_final = compute_engagement_tw2(df_final)
    print(i, a, b)
    df_final.to_parquet(f'{agg}intermediate/may_batch2{i}.parquet.gzip', 
                        compression = 'gzip')

  0%|          | 0/2 [00:00<?, ?it/s]

5 15000 18000


 50%|█████     | 1/2 [35:59<35:59, 2159.38s/it]

6 18000 21000


100%|██████████| 2/2 [52:17<00:00, 1568.71s/it]


In [4]:
country = 'SA'
path_tw, base, rand, baseline, agg, agg_base = get_path(country, 'april')
path_tw1 = f'../../data/03-experiment/{country}/treatment/followers/00-raw/tweets_batch2/may/'
onlyfiles = [f for f in listdir(path_tw1) if isfile(join(path_tw1, f))]
onlyfiles = [f.replace('.parquet', '') for f in onlyfiles]
onlyfiles.sort()
len(onlyfiles)

19178